In [ ]:
"""
NHS MCDA for patient pathway prioritization

This script implements a small, flexible Multi-Criteria Decision Analysis (MCDA)
for ranking patients within each Speciality (department).

Primary rule from user:
  1) Group (primary key): Speciality
  2) Within each Speciality, rank using criteria in this order (but implemented
     as a weighted MCDA so weights can be tuned):
       - Complexity (numerical) — larger => higher priority
       - Acuity (1..5, higher = worse) — larger => higher priority
       - VitalsTrend (categorical) — priority order: Deteriorating > Stable > Improving

Features:
  - Normalises numeric values (optionally within each Speciality)
  - Maps VitalsTrend to an ordinal score
  - Allows adjustable weights for each criterion
  - Handles missing data sensibly
  - Returns a ranked DataFrame per Speciality and an overall ordering (Speciality groups preserved)

Usage example included at bottom.
"""

from typing import Dict, Any
import pandas as pd
import numpy as np


DEFAULT_VITALS_ORDER = {
    'Deteriorating': 1.0,
    'Stable': 0.5,
    'Improving': 0.0
}


def normalize_series(s: pd.Series) -> pd.Series:
    """Min-max normalize a pandas Series to [0,1]. If constant, returns 0.5 for all.
    NaNs are left as NaN.
    """
    valid = s.dropna()
    if valid.empty:
        return s
    mn = valid.min()
    mx = valid.max()
    if mn == mx:
        # constant series; return 0.5 for known values
        out = s.copy()
        out.loc[s.notna()] = 0.5
        return out
    return (s - mn) / (mx - mn)


def compute_mcda_scores(
    df: pd.DataFrame,
    weights: Dict[str, float] = None,
    vitals_map: Dict[str, float] = None,
    normalize_within_Speciality: bool = True,
    patient_id: str = 'pseudo_patient_id',
    age_col: str = 'age',
    gender_col: str = 'sex',
    complexity_col: str = 'Complexity',
    acuity_col: str = 'Acuity',
    primarydiagnosis_col: str = 'Primary Diagnosis Summary',
    department_col: str = 'Speciality',
    vitals_col: str = 'Vitals Trend',
    waiting_col: str = 'Waiting Time (days)',
    admission_col: str = 'Time since Admission (days)',
    nextAction_col: str = 'nextAction',
    blocker_col: str = 'blocker',
    dischargedependence_col: str = 'Discharge Dependence'
) -> pd.DataFrame:

    df = df.copy()
    required = ['Speciality', complexity_col, acuity_col, waiting_col]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Input df is missing required columns: {missing}")

    # Ensure all expected keys are present (missing keys get 0.0)
    for k in ['complexity', 'acuity', 'waiting']:
        weights.setdefault(k, 0.0)

    w_total = sum(weights.values())
    if w_total == 0:
        raise ValueError('Sum of weights must be > 0')
    weights = {k: v / w_total for k, v in weights.items()}

    if vitals_map is None:
        vitals_map = DEFAULT_VITALS_ORDER

    df['_vitals_score_raw'] = df[vitals_col].map(vitals_map)
    if df['_vitals_score_raw'].isna().any():
        known_median = df['_vitals_score_raw'].median(skipna=True)
        df['_vitals_score_raw'].fillna(known_median, inplace=True)

    if normalize_within_Speciality:
        norm_complexity = df.groupby('Speciality')[complexity_col].transform(lambda s: normalize_series(s))
        norm_acuity = df.groupby('Speciality')[acuity_col].transform(lambda s: normalize_series(s))
        norm_wait = df.groupby('Speciality')[waiting_col].transform(lambda s: normalize_series(s))
        norm_admit = df.groupby('Speciality')[admission_col].transform(lambda s: normalize_series(s))
    else:
        norm_complexity = normalize_series(df[complexity_col])
        norm_acuity = normalize_series(df[acuity_col])
        norm_wait = normalize_series(df[waiting_col])
        norm_admit = normalize_series(df[admission_col])

    df['_norm_complexity'] = df.groupby('Speciality')[complexity_col].transform(lambda s: normalize_series(s)).fillna(0.5)
    df['_norm_acuity'] = df.groupby('Speciality')[acuity_col].transform(lambda s: normalize_series(s)).fillna(0.5)
    df['_norm_wait'] = df.groupby('Speciality')[waiting_col].transform(lambda s: normalize_series(s)).fillna(0.5)
    df['_norm_admit'] = df.groupby('Speciality')[admission_col].transform(lambda s: normalize_series(s)).fillna(0.5)

    df['MCDA_score'] = (
        weights['complexity'] * df['_norm_complexity'] +
        weights['acuity'] * df['_norm_acuity'] +
        weights['vitals'] * df['_vitals_score_raw'] +
        weights['waiting'] * df['_norm_wait'] +
        weights['admission'] * df['_norm_admit']
    )

    df['mcda_rank_within_Speciality'] = df.groupby('Speciality')['MCDA_score'].rank(method='first', ascending=False).astype(int)

    df['tie_breaker'] = df[[acuity_col, complexity_col, vitals_col]].apply(
        lambda row: f"Acuity={row[acuity_col]}|Complexity={row[complexity_col]}|Vitals={row[vitals_col]}", axis=1
    )

    df.drop(columns=['_vitals_score_raw', '_norm_complexity', '_norm_acuity', '_norm_wait', '_norm_admit'], inplace=True)

    return df


def rank_within_all_specialties(df_out: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a DataFrame ordered first by Speciality (alphabetical) and then by mcda_rank_within_Speciality.
    If you prefer a different Speciality order, reorder 'Speciality' before calling this function.
    """
    df = df_out.copy()
    df.sort_values(['Speciality', 'mcda_rank_within_Speciality'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df


# -------------------------------
# Example usage (run as script or import functions)
# -------------------------------
if __name__ == '__main__':
    # Load ScenarioA data
    from pathlib import Path
    import pandas as pd

    data_dir = Path('../data/scenarioA/')
    paths = {
        "current": data_dir / "ScenarioA_patients_current.csv",
        "coming": data_dir / "ScenarioA_patients_coming.csv",
        "historic": data_dir / "ScenarioA_patients_historic.csv"
    }

    dfs = {k: pd.read_csv(v) for k, v in paths.items()}
    for name, df in dfs.items():
        df.columns = [c.strip() for c in df.columns]
        dfs[name] = df

    current = dfs["current"].copy()
    coming = dfs["coming"].copy()
    historic = dfs["historic"].copy()

    # Choose which dataset to run: 'current', 'coming', or 'historic'
    which = 'coming'
    input_df = dfs[which].copy()

    if which == 'current':
        weights = {
            'complexity': 0.20,
            'acuity': 0.30,
            'vitals': 0.30,
            'waiting': 0.10,
            'admission': 0.10
        }
    elif which == 'coming':
        weights = {
            'complexity': 0.30,
            'acuity': 0.50,
            'waiting': 0.20,
        }
    # elif which == 'historic':
    #     weights = {
    #         'complexity': 0.25,
    #         'acuity': 0.25,
    #         'vitals': 0.25,
    #         'waiting': 0.125,
    #         'admission': 0.125
    #     }

    ranked = compute_mcda_scores(input_df, weights=weights, normalize_within_Speciality=True)
    ordered = rank_within_all_specialties(ranked)

    save_name = f'mcda_ranked_patients_{which}.csv'
    ordered.to_csv(save_name, index=False)
    print(ordered.head())


KeyError: 'Column not found: Time since Admission (days)'